In [ ]:
import tensorflow as tf 

In [ ]:
def modify_embeddings(tensor):
    batch_size, seq_len, embed_dim = tensor.shape
    
    # Chu kỳ tuần hoàn các số 0, +3.14 * -1/2, -3.14 * -1/2, và /3.14
    pi = 3.14
    a = tf.sqrt(pi)
    b = tf.sqrt(pi)
    values = [0.0, a, b, pi]
    num_values = len(values)
    
    for i in range(batch_size):
        i = tf.cast(i, tf.int32)
        for j in range(0, seq_len, 3):  # Lặp qua mỗi 3 chỉ số
            idx = tf.cast((tf.cast(j, tf.int32) // 3) % num_values, tf.int32)  # Xác định phép toán cần thực hiện
            if idx == 0:
                tensor = tf.tensor_scatter_nd_update(tensor, [[i, j, k] for k in range(embed_dim)], [0] * embed_dim)
            elif idx == 1:
                tensor = tf.tensor_scatter_nd_update(tensor, [[i, j, k] for k in range(embed_dim)], tensor[i, j, :] + values[idx])
            elif idx == 2:
                tensor = tf.tensor_scatter_nd_update(tensor, [[i, j, k] for k in range(embed_dim)], tensor[i, j, :] - values[idx])
            elif idx == 3:
                tensor = tf.tensor_scatter_nd_update(tensor, [[i, j, k] for k in range(embed_dim)], tensor[i, j, :] / values[idx])
    
    return tensor

# Ví dụ tensor đầu vào với kích thước [batch_size, seq_len, embed_dim]
tensor = tf.random.uniform((2, 10, 5))  # Bạn có thể thay đổi kích thước này theo nhu cầu

# Gọi hàm và in ra tensor đã được thay đổi
modified_tensor = modify_embeddings(tensor)
print(modified_tensor)

In [ ]:
import tensorflow as tf 

In [ ]:
import tensorflow as tf

class Attention:
    def __init__(self):
        pass

    def transform_tensor(self, q, params, step):
        """
        Biến đổi tensor q trên các tham số bằng cách thực hiện các phép tính trên tensor q với 
        các tham số và phép tính sẽ cách nhau một khoảng step.

        Args:
            q (tf.Tensor): tensor đầu vào, kích thước (batch_size, sequence_length, feature_dim).
            params (list): danh sách các tham số để thực hiện phép toán.
            step (int): khoảng cách giữa các phép toán trên tensor q.
        
        Returns:
            tf.Tensor: tensor sau khi đã biến đổi.
        """
        param_length = len(params)
        q_transformed = tf.identity(q)
        
        batch_size, sequence_length, feature_dim = q.shape
        
        for i in range(sequence_length):
            # Chọn phép toán dựa trên chỉ số hiện tại và số lượng tham số
            param_index = (i // step) % param_length
            param_value = params[param_index]
            
            # Tạo một tensor để cộng giá trị tham số vào vị trí cần thiết
            add_tensor = tf.constant(param_value, shape=[batch_size, 1, feature_dim], dtype=q.dtype)
            
            # Cộng giá trị của tham số vào phần tử hiện tại của q
            mask = tf.concat([tf.zeros([batch_size, i, feature_dim], dtype=q.dtype),
                              add_tensor,
                              tf.zeros([batch_size, sequence_length - i - 1, feature_dim], dtype=q.dtype)], axis=1)
            q_transformed = q_transformed + mask
        
        return q_transformed

# Ví dụ sử dụng
q = tf.constant([
    [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]],
    [[9.0, 10.0], [11.0, 12.0], [13.0, 14.0], [15.0, 16.0]]
], dtype=tf.float32)

params = [0, 1, 2, 4]
step = 2

attention = Attention()
q_transformed = attention.transform_tensor(q, params, step)
print(q_transformed)
attention = Attention()
q_transformed = attention.transform_tensor(q, params, step)
print(q_transformed)


In [ ]:
q = tf.random.uniform((2, 10, 5)) 
q_transformed = tf.reshape(q, shape=(2, -1))
q_transformed.shape, q[:, :, 1]

In [ ]:
def transform_tensor(q):
    q_transformed = tf.identity(q)
    batch_size, seq_length, feature_dim = q.shape

    q_transformed = tf.reshape(q_transformed, shape=(batch_size, -1))
    length, dim = q_transformed.shape

    for s in range(length):
        for i in range(0, dim, 3):
            idx = i // 3 % 4
            if idx == 0:
                q_transformed = tf.tensor_scatter_nd_update(q_transformed, [[s, i]],
                                                 [tf.multiply(q_transformed[s, i], 0.0)])
            elif idx == 1:
                q_transformed = tf.tensor_scatter_nd_update(q_transformed, [[s, i]],
                                                 [tf.add(q_transformed[s, i], tf.math.sqrt(3.14))])
            elif idx == 2:
                q_transformed = tf.tensor_scatter_nd_update(q_transformed, [[s, i]],
                                                 [tf.subtract(q_transformed[s, i], tf.math.sqrt(3.14))])
            elif idx == 3:
                q_transformed = tf.tensor_scatter_nd_update(q_transformed, [[s, i]],
                                                 [tf.divide(q_transformed[s, i], tf.cast(3.14, tf.float32))])

    return tf.reshape(q_transformed, shape=(batch_size, -1, feature_dim))

# Ví dụ sử dụng
q = tf.random.uniform((2, 10, 5))
q_transformed = transform_tensor(q)
q_transformed

In [ ]:

import tensorflow as tf 

def transform_tensor(q):

    params = tf.cast([0, 3.14, 3.14, 3.14**0.5], tf.float32) 
    num_values = len(params)
    q_transformed = tf.identity(q)
    batch_size, seq_length, feature_dim = q.shape


    q_transformed = tf.reshape(q_transformed , shape=(batch_size, -1))
    
    step = 3

    for s in range (batch_size):
        for i in range (0, seq_length, 3):
            idx = i // 3 % num_values
            if idx == 0: 
                q_transformed = tf.multiply(q_transformed[s, i, :], 0)

            elif idx == 1:
                q_transformed = tf.add(q_transformed[s, i, :], tf.math.sqrt(3.14))
                
            elif idx == 2:
                q_transformed = tf.subtract(q_transformed[s, i, :], tf.math.sqrt(3.14))

            elif idx == 3:
                q_transformed = tf.devide(q_transformed[s, i, :], tf.cast(3.14, tf.float32))
        
        
    return q_transformed


# Ví dụ sử dụng
q = tf.random.uniform((2, 10, 5)) 



q_transformed = transform_tensor(q)
q, q_transformed

In [ ]:
def transform_tensor(self, q, params, step):
    param_length = len(params)
    q_transformed = tf.identity(q)
    
    batch_size, sequence_length, feature_dim = q.shape

    operations = [
        tf.multiply,   # Phép nhân (*)
        tf.add,        # Phép cộng (+)
        tf.subtract,   # Phép trừ (-)
        tf.divide      # Phép chia (/)
    ]
    
    for i in range(0, sequence_length, step):
        if i + step < sequence_length:
            op_index = i % len(operations)
            param_index = i % param_length
            param_value = params[param_index]
            operation = operations[op_index]
            
            current_slice = q_transformed[:, i + step, :]
            result_slice = operation(current_slice, param_value)
            
            q_transformed = tf.concat([q_transformed[:, :i + step, :], 
                                       tf.expand_dims(result_slice, 1), 
                                       q_transformed[:, i + step + 1:, :]], axis=1)
        else:
            break  # Dừng khi hết seq_length
    
    return q_transformed

# Ví dụ sử dụng
q = tf.constant([
    [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0], [9.0, 10.0], [11.0, 12.0], [13.0, 14.0], [15.0, 16.0], [17.0, 18.0], [19.0, 20.0]]
], dtype=tf.float32)

params = [0, 3.14, 3.14, 3.14**0.5]
step = 3

attention = Attention()
q_transformed = attention.transform_tensor(q, params, step)
print(q_transformed)


In [ ]:
import torch  
from torch.nn 

In [ ]:
import torch
import torch.nn.functional as F

def transform_tensor(tensor):
    q_transformed = tensor.clone()
    batch_size, seq_length, feature_dim = tensor.shape

    q_transformed = q_transformed.view(batch_size, -1)
    length, dim = q_transformed.shape

    for s in range(length):
        for i in range(0, dim, 20):
            idx = (i // 20) % 4
            if idx == 0:
                q_transformed[s, i] = q_transformed[s, i] * 0.0
            elif idx == 1:
                q_transformed[s, i] = q_transformed[s, i] + torch.sqrt(torch.tensor(3.14))
            elif idx == 2:
                q_transformed[s, i] = q_transformed[s, i] - torch.sqrt(torch.tensor(3.14))
            elif idx == 3:
                q_transformed[s, i] = q_transformed[s, i] / 3.14

    return q_transformed.view(batch_size, seq_length, feature_dim)


In [ ]:
import torch
import torch.nn.functional as F

def transform_tensor(tensor):
    q_transformed = tensor.clone()
    batch_size, seq_length, feature_dim = tensor.shape

    q_transformed = q_transformed.view(batch_size, -1)
    length, dim = q_transformed.shape

    sqrt_3_14 = torch.sqrt(torch.tensor(3.14))

    for s in range(length):
        for i in range(0, dim, 20):
            idx = (i // 20) % 4
            if idx == 0:
                q_transformed[s, i] = 0.0
            elif idx == 1:
                q_transformed[s, i] += sqrt_3_14
            elif idx == 2:
                q_transformed[s, i] -= sqrt_3_14
            elif idx == 3:
                q_transformed[s, i] /= 3.14

    return q_transformed.view(batch_size, seq_length, feature_dim)


In [ ]:
import torch

a = torch.tensor([0, 1, 2, 3])
result = torch.exp(a) # 
print(result)


In [ ]:
import torch
import math

# Khởi tạo tensor với kích thước 256
tensor = torch.randn(size=(128, 90, 256))

# Chia tensor thành 8 phần (mỗi phần có kích thước 32)
splits = torch.split(tensor, [128, 90, 32])

# Nhân phần thứ 5 với căn 3.14
splits[4] = splits[4] * math.sqrt(3.14)

# Kết hợp lại thành tensor ban đầu
result_tensor = torch.cat(splits)

print(result_tensor)
